# 실습설명
화장품 이커머스 데이터 분석을 통해, 고객의 이탈율을 낮추고 구매 전환율을 높이는 것이 목표

# 데이터 설명

- event_time : 이벤트가 발생한 시간
- event_type : 이벤트 유형(구매)
- product_id : 제품ID
- category_id : 제품의 카테고리 ID
- category_code : 제품의 카테고리 코드 (의미있는 카테고리의 경우 존재)
- brand : 브랜드 이름 (소문자, 생략될 수 있음)
- price : 제품의 가격
- user_id : 사용자 ID
- user_session : 사용자 세션 ID(임시, 세션마다 변경됨)

In [31]:
import pandas as pd
df  = pd.read_csv("/Users/jangminseul/Desktop/ecommerce/data/raw/cosmetic.csv")
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


### EDA

In [32]:
print(df.shape) # 데이터 개수
print(df.info()) # 컬럼별 타입

(3533286, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533286 entries, 0 to 3533285
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 242.6+ MB
None


In [33]:
df_raw = df.copy() # 원본 보존
df_clean = df.copy() # 전처리용


In [34]:
df_clean['event_time'] = pd.to_datetime(df_clean['event_time']) # 데이터 타입 변환

In [35]:
df_clean['event_time'].dtype

datetime64[ns, UTC]

In [36]:
df_clean.duplicated().sum() # 중복데이터 확인

183860

중복 데이터 원인
- 같은 행동이 동시에 여러번 기록
- 초 단위 timestamp
- 세션 로그 특성

In [37]:
df_clean.duplicated().sum()/len(df_clean) # 중복 데이터 비율 확인

0.052036546149957855

In [38]:
df_clean[df_clean.duplicated()].head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
37,2019-12-01 00:01:52+00:00,cart,5700046,1487580009286598681,NaN,runail,0.40,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
55,2019-12-01 00:02:49+00:00,cart,5693501,1487580009445982239,NaN,NaN,0.60,579751441,8de492d7-0937-47ae-be2c-a7615aec2b0d
61,2019-12-01 00:03:08+00:00,remove_from_cart,5859482,1487580005671109489,NaN,masura,1.60,561162056,39cf2227-03ed-421e-9615-7814b9b3c5e6
62,2019-12-01 00:03:08+00:00,remove_from_cart,5839671,1487580005671109489,NaN,masura,2.19,561162056,39cf2227-03ed-421e-9615-7814b9b3c5e6
66,2019-12-01 00:03:10+00:00,remove_from_cart,5859482,1487580005671109489,NaN,masura,1.60,561162056,39cf2227-03ed-421e-9615-7814b9b3c5e6


In [39]:
df_clean.duplicated(subset=['user_id','event_time','product_id','event_type']).sum()

185220

In [40]:
df_clean.drop_duplicates(subset=['user_id','event_time','product_id','event_type'])  # 어떤 종류인지 확인
df_clean = df_clean.drop_duplicates(subset=['user_id','event_time','product_id','event_type']) # 같은 사용자가, 같은 시점에, 같은 상품에 대해 같은 행동을 여러번 기록 한 경우로 바라보고 분석하기 위함

In [41]:
print(len(df_clean))
df_clean.duplicated(subset=['user_id','event_time','product_id','event_type']).sum() # 제거확인

3348066


0

In [42]:
print(df_clean.isnull().sum()) # 결측치 확인
(df_clean.isnull().sum()/len(df_clean))*100  # 결측치 비율 확인

event_time             0
event_type             0
product_id             0
category_id            0
category_code    3291359
brand            1427640
price                  0
user_id                0
user_session         714
dtype: int64


event_time        0.000000
event_type        0.000000
product_id        0.000000
category_id       0.000000
category_code    98.306276
brand            42.640736
price             0.000000
user_id           0.000000
user_session      0.021326
dtype: float64

In [52]:
df_clean = df_clean.drop('category_id', axis= 1)

In [ ]:
df_clean = df_clean

### 데이터 분석

In [43]:
df_clean['price'].describe() # 수치 분포 확인

count    3.348066e+06
mean     9.079971e+00
std      2.024615e+01
min     -7.937000e+01
25%      2.060000e+00
50%      4.270000e+00
75%      7.140000e+00
max      3.277800e+02
Name: price, dtype: float64

In [44]:
df_clean[df_clean['price'] < 0].shape # (17,9)
(df_clean['price'] < 0).mean()
df_clean[df_clean['price'] < 0].head()
df_clean[df_clean['price'] < 0]['event_type'].value_counts() # 17
# import matplotlib.pyplot as plt
# plt.hist(df['price'], bins = 50)
# plt.show()

event_type
purchase    17
Name: count, dtype: int64

In [45]:
df_clean = df_clean[df_clean['price'] >= 0]

In [46]:
df_clean['event_type'].value_counts() 
# 퍼널 구조 view -> cart -> purchase
# 중간 이탈 remove_from_cart

event_type
view                1726861
cart                 911641
remove_from_cart     496498
purchase             213049
Name: count, dtype: int64

In [29]:
# 단계별 전환율
view = 1728331
cart = 927124
remove_from_cart = 664655
purchase = 213176

view_to_cart = cart / view
cart_to_purchase = purchase / cart
view_to_purchase = purchase / view

view_to_cart,cart_to_purchase,view_to_purchase

(0.5364273394390311, 0.22993256565464815, 0.12334211444451323)

In [47]:
# remove_from_cart 분석
cart_count = (df_clean['event_type'] == 'cart').sum()
remove_count = (df_clean['event_type'] == 'remove_from_cart').sum()
remove_rate = (remove_count/cart_count)
cart_count, remove_count, remove_rate

(911641, 496498, 0.544620086196211)

In [51]:
# 가격대별 전환율 분석
df_processed = df_clean.copy()
df_processed['price_bin'] = pd.qcut(df_processed['price'],q=4)  # 가격 데이터 4등분
df_processed['price_bin'].value_counts()



price_bin
(2.06, 4.27]      838318
(-0.001, 2.06]    837288
(4.27, 7.14]      836360
(7.14, 327.78]    836083
Name: count, dtype: int64